In [1]:
using Geodesy
include("mathies.jl")

In [2]:
myspot = LLA( 30.228711, 92.027100 )
ced404 = LLA( 30.228678, 92.027272 )
ber266 = LLA( 30.204461, 92.031397 )
# if I wanted to do a 3-D transformation; to put into cartesian coordinates do 
myspotECEF = ECEF( myspot, wgs84 )

LLA(lat=30.204461°, lon=92.031397°, alt=0.0)

In [15]:
""" modgin treats space as an expanding or contracting series of pentagonally-arranged, overlapping circles 

    it divides/expands a circle into six new circles, with the centerpoint of the center circle remaining the same,
    and new centerpoints for the five new circles at predermined distances and angles from the original centerpoint

    for this reason, mapping space in modgin relies heavily on the use of the "ENU" (East, North, Up) coordinate system; 
    once the first 12 coordinates for earth's surface are established, the coordinates for every lower level are relative
    those positions

    The SESpaces Coordinate System specifies twelve points on the globe as "beads," or centerpoints for six regions 
    each in the northern and southern hemispheres. 

    These regions 
"""    

" modgin treats space as an expanding or contracting series of pentagonally-related circles (as spheres is hopefully doablef);\n\n    it divides/expands a circle into six new circles, with the centerpoint of the center circle remaining the same,\n    and new centerpoints for the five new circles at predermined distances and angles from the original centerpoint\n\n    for this reason, mapping space in modgin relies heavily on the use of the \"ENU\" (East, North, Up) coordinate system; \n    once the first 12 coordinates for earth's surface are established, the coordinates for every lower level are relative\n    those positions\n\n    \n\n"

In [10]:
hafu / hafe
rad2deg(pio5)

36.0